In [31]:
import pandas as pd
import os
import numpy as np
import pygeohash as pgh

In [32]:
os.chdir('C:/Users/wissam_T/Desktop/5th/2nd semester/dm/h.w 1/project')

In [33]:
from project.functions import helper_functions as hf

In [34]:
df_merged = pd.read_csv('datasets/Merged_trips_with_stations.csv')

In [35]:
df_merged.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'member_casual', 'duration_min', 'geometry_start',
       'geometry_end', 'Zone_start_zone_name', 'Zone_start_GIS_ID',
       'Zone_start_OBJECTID', 'Zone_start_geometry', 'Zone_end_zone_name',
       'Zone_end_GIS_ID', 'Zone_end_OBJECTID', 'Zone_end_geometry', 'date',
       'Weather_temp', 'Weather_windspeedmean', 'Weather_sunrise',
       'Weather_sunset', 'Weather_conditions', 'Bikeshare_NAME_start',
       'Bikeshare_STATION_TYPE_start', 'Bikeshare_CAPACITY_start',
       'Bikeshare_REGION_ID_start', 'Bikeshare_REGION_NAME_start',
       'Bikeshare_geometry_start', 'Bikeshare_NAME_end',
       'Bikeshare_STATION_TYPE_end', 'Bikeshare_CAPACITY_end',
       'Bikeshare_REGION_ID_end', 'Bikeshare_REGION_NAME_end',
       'Bikeshare_geometry_end', 'StartStation_NAME', 'EndStation_NAME'],
      dtype='object')

In [36]:
print(df_merged['Bikeshare_CAPACITY_start'].isna().sum())
print(df_merged['Bikeshare_CAPACITY_end'].isna().sum())

0
28


In [37]:
df_merged.shape

(97, 40)

In [38]:
df_merged_copy = df_merged.copy()

In [39]:
# First, remove the "POINT (" and ")" parts, then split
df_modified = pd.DataFrame(df_merged_copy)
df_modified['start_longitude'] = df_modified['Bikeshare_geometry_start'].str.extract(r'POINT \((-?\d+\.\d+)')[0].astype(float)
df_modified['start_latitude'] = df_modified['Bikeshare_geometry_start'].str.extract(r'POINT \(-?\d+\.\d+ (\d+\.\d+)\)')[0].astype(float)

df_modified['end_longitude'] = df_modified['Bikeshare_geometry_end'].str.extract(r'POINT \((-?\d+\.\d+)')[0].astype(float)
df_modified['end_latitude'] = df_modified['Bikeshare_geometry_end'].str.extract(r'POINT \(-?\d+\.\d+ (\d+\.\d+)\)')[0].astype(float)


In [40]:
df_modified['rounded_start_lat'] = df_modified['start_latitude'].round(5)
df_modified['rounded_start_lon'] = df_modified['start_longitude'].round(5)

ref_from_start = df_modified[
    df_modified['end_station_name'].notna() &
    df_modified['start_latitude'].notna() &
    df_modified['start_longitude'].notna()
    ][
    ['rounded_start_lat', 'rounded_start_lon', 
     'end_station_name', 'end_station_id',
     'end_latitude', 'end_longitude',
     'Bikeshare_NAME_end', 'Bikeshare_STATION_TYPE_end',
     'Bikeshare_CAPACITY_end', 'Bikeshare_REGION_ID_end',
     'Bikeshare_REGION_NAME_end', 'Bikeshare_geometry_end',
     'Zone_end_zone_name', 'Zone_end_GIS_ID',
     'Zone_end_OBJECTID', 'Zone_end_geometry',
     'EndStation_NAME']
].drop_duplicates(subset=['rounded_start_lat', 'rounded_start_lon'])

ref_from_start.set_index(['rounded_start_lat', 'rounded_start_lon'], inplace=True)

end_cols_to_fill = [
    'end_station_name', 'end_station_id',
    'end_latitude', 'end_longitude',
    'Bikeshare_NAME_end', 'Bikeshare_STATION_TYPE_end',
    'Bikeshare_CAPACITY_end', 'Bikeshare_REGION_ID_end',
    'Bikeshare_REGION_NAME_end', 'Bikeshare_geometry_end',
    'Zone_end_zone_name', 'Zone_end_GIS_ID',
    'Zone_end_OBJECTID', 'Zone_end_geometry',
    'EndStation_NAME'
]

for col in end_cols_to_fill:
    df_modified[col] = df_modified.apply(lambda row: hf.fill_from_start_coords(row, col, ref_from_start), axis=1)

df_modified.drop(columns=['rounded_start_lat', 'rounded_start_lon'], inplace=True)


In [41]:
def assign_size_category(cap):
    if cap < low_threshold:
        return 'Small'
    elif cap < high_threshold:
        return 'Medium'
    else:
        return 'Large'

In [42]:
start_stations = df_merged[['start_station_id', 'Bikeshare_CAPACITY_start']].rename(
    columns={'start_station_id': 'station_id', 'Bikeshare_CAPACITY_start': 'capacity'}
)

end_stations = df_merged[['end_station_id', 'Bikeshare_CAPACITY_end']].rename(
    columns={'end_station_id': 'station_id', 'Bikeshare_CAPACITY_end': 'capacity'}
)

# Combine both into one list of stations
stations = pd.concat([start_stations, end_stations])

# Remove nulls and duplicates (in case some stations appear twice)
stations = stations.drop_duplicates(subset='station_id')

# Step 2: Calculate thresholds (33rd and 66th percentiles)
low_threshold = np.percentile(stations['capacity'], 33)
high_threshold = np.percentile(stations['capacity'], 66)

# Step 4: Apply the function to create the new column
stations['station_size'] = stations['capacity'].apply(assign_size_category)

# Now `stations` has: station_id, capacity, and station_size
print(stations.head())


    station_id  capacity station_size
0      31519.0      15.0        Large
7      31651.0      19.0        Large
9      31939.0      17.0        Large
10     30200.0      27.0        Large
11     32265.0      12.0        Large


In [43]:
# Merge back to the original df for start stations
df_with_sizes = df_merged_copy.merge(
    stations[['station_id', 'station_size']],
    left_on='start_station_id',
    right_on='station_id',
    how='left'
).rename(columns={'station_size': 'start_station_size'}).drop(columns=['station_id'])

# Merge back again for end stations
df_with_sizes = df_with_sizes.merge(
    stations[['station_id', 'station_size']],
    left_on='end_station_id',
    right_on='station_id',
    how='left'
).rename(columns={'station_size': 'end_station_size'}).drop(columns=['station_id'])


In [44]:
df_with_sizes.shape

(97, 42)

In [45]:
df_merged_copy.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'member_casual', 'duration_min', 'geometry_start',
       'geometry_end', 'Zone_start_zone_name', 'Zone_start_GIS_ID',
       'Zone_start_OBJECTID', 'Zone_start_geometry', 'Zone_end_zone_name',
       'Zone_end_GIS_ID', 'Zone_end_OBJECTID', 'Zone_end_geometry', 'date',
       'Weather_temp', 'Weather_windspeedmean', 'Weather_sunrise',
       'Weather_sunset', 'Weather_conditions', 'Bikeshare_NAME_start',
       'Bikeshare_STATION_TYPE_start', 'Bikeshare_CAPACITY_start',
       'Bikeshare_REGION_ID_start', 'Bikeshare_REGION_NAME_start',
       'Bikeshare_geometry_start', 'Bikeshare_NAME_end',
       'Bikeshare_STATION_TYPE_end', 'Bikeshare_CAPACITY_end',
       'Bikeshare_REGION_ID_end', 'Bikeshare_REGION_NAME_end',
       'Bikeshare_geometry_end', 'StartStation_NAME', 'EndStation_NAME'],
      dtype='object')

In [46]:
df_modified.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,member_casual,duration_min,...,Bikeshare_CAPACITY_end,Bikeshare_REGION_ID_end,Bikeshare_REGION_NAME_end,Bikeshare_geometry_end,StartStation_NAME,EndStation_NAME,start_longitude,start_latitude,end_longitude,end_latitude
0,748A93D7DE8A41CD,classic_bike,2024-01-25 15:49:59,2024-01-25 15:52:35,1st & O St NW,31519,1st & L St NW,31677.0,member,2.600000,...,19.0,42.0,DCA-CABI,POINT (-77.011987 38.903819),1st & O St NW,1st & L St NW,-77.012365,38.908643,-77.011987,38.903819
1,75CBFD136F06305B,classic_bike,2024-01-02 16:44:58,2024-01-02 16:53:25,1st & O St NW,31519,4th & College St NW,31138.0,member,8.450000,...,15.0,42.0,DCA-CABI,POINT (-77.018135 38.921233),1st & O St NW,4th & College St NW,-77.012365,38.908643,-77.018135,38.921233
2,0536C9720F87E04C,classic_bike,2024-01-24 15:40:15,2024-01-24 15:43:55,1st & O St NW,31519,1st & L St NW,31677.0,member,3.666667,...,19.0,42.0,DCA-CABI,POINT (-77.011987 38.903819),1st & O St NW,1st & L St NW,-77.012365,38.908643,-77.011987,38.903819
3,9E17390C218783B5,classic_bike,2024-01-04 15:35:00,2024-01-04 15:37:35,1st & O St NW,31519,1st & L St NW,31677.0,member,2.583333,...,19.0,42.0,DCA-CABI,POINT (-77.011987 38.903819),1st & O St NW,1st & L St NW,-77.012365,38.908643,-77.011987,38.903819
4,00727D0E773CDFF7,electric_bike,2024-01-05 12:27:58,2024-01-05 12:35:40,1st & O St NW,31519,10th & G St NW,31274.0,casual,7.700000,...,23.0,42.0,DCA-CABI,POINT (-77.026235 38.898243),1st & O St NW,10th & G St NW,-77.012365,38.908643,-77.026235,38.898243


In [47]:
shopping_centers = np.array([
    [38.9009, -77.0260],   # CityCenterDC
    [38.8971, -77.0064],   # Union Station
    [38.9057, -77.0631],   # Georgetown Park
    [38.8631, -77.0599],   # Pentagon City
    [38.8765, -77.0316],   # The Wharf
    [38.8741, -77.0028],   # Capitol Riverfront
    [38.9613, -77.0840],   # Friendship Heights
])

In [48]:
proximity = 300
df_modified['start_near_any_mall'] = df_modified.apply(
    lambda row: int(hf.distance_to_closest_mall(row['start_latitude'], row['start_longitude'], shopping_centers) <= proximity),
    axis=1
)

df_modified['end_near_any_mall'] = df_modified.apply(
    lambda row: int(hf.distance_to_closest_mall(row['end_latitude'], row['end_longitude'], shopping_centers) <= proximity),
    axis=1
)

In [49]:
count_of_ones = df_modified['start_near_any_mall'].eq(1).sum()

print(f"Number of 1s in '{'start_near_any_mall'}': {count_of_ones}")

count_of_ones = df_modified['end_near_any_mall'].eq(1).sum()

print(f"Number of 1s in '{'end_near_any_mall'}': {count_of_ones}")

Number of 1s in 'start_near_any_mall': 2
Number of 1s in 'end_near_any_mall': 17


In [54]:
df_modified['end_distance_to_closest_mall_m'] = df_modified.apply(
    lambda row: hf.distance_to_closest_mall(row['end_latitude'], row['end_longitude'], shopping_centers),
    axis=1
)

In [55]:
df_modified.to_csv('datasets/modified.csv', index=False)

PermissionError: [Errno 13] Permission denied: 'datasets/modified.csv'

In [56]:
null_counts = df_modified.isnull().sum()

# Calculate total number of rows in the DataFrame
total_rows = len(df_modified)

# Calculate percentage of null values for all columns
# Use .apply(lambda x: f"{x:.2f}%") for formatting if you want to store as string
null_percentages = (null_counts / total_rows) * 100

# Create a new DataFrame to store these results
null_summary_df = pd.DataFrame({
    'Null Count': null_counts,
    'Null Percentage': null_percentages
})

# Optional: Sort by Null Count or Null Percentage to see most affected columns first
null_summary_df = null_summary_df.sort_values(by='Null Count', ascending=False)

rows_with_nulls_df = df_modified[df_modified.isnull().any(axis=1)]

null_summary_df

,Null Count,Null Percentage
Zone_start_OBJECTID,31,31.958763
Zone_start_geometry,31,31.958763
Zone_start_zone_name,31,31.958763
Zone_start_GIS_ID,31,31.958763
Zone_end_geometry,22,22.680412
Zone_end_OBJECTID,22,22.680412
Zone_end_GIS_ID,22,22.680412
Zone_end_zone_name,22,22.680412
Bikeshare_NAME_end,5,5.154639
Bikeshare_STATION_TYPE_end,5,5.154639


In [57]:
precision = 6

df_modified['start_geohash'] = df_modified.apply(
    lambda row: pgh.encode(row['start_latitude'], row['start_longitude'], precision=precision)
    if not np.isnan(row['start_latitude']) and not np.isnan(row['start_longitude']) else np.nan,
    axis=1
)

df_modified['end_geohash'] = df_modified.apply(
    lambda row: pgh.encode(row['end_latitude'], row['end_longitude'], precision=precision)
    if not np.isnan(row['end_latitude']) and not np.isnan(row['end_longitude']) else np.nan,
    axis=1
)

In [58]:
daily_trip_counts = df_modified.groupby(['start_geohash', 'date']).size().reset_index(name='trip_count')

avg_trips_per_geohash = daily_trip_counts.groupby('start_geohash')['trip_count'].mean().reset_index()
avg_trips_per_geohash.rename(columns={'trip_count': 'avg_daily_trips'}, inplace=True)

quantiles = avg_trips_per_geohash['avg_daily_trips'].quantile([0.22, 0.88])
low_thresh = quantiles[0.22]
high_thresh = quantiles[0.88]

In [60]:
def categorize_traffic(avg):
    if avg <= low_thresh:
        return 'Low Traffic'
    elif avg <= high_thresh:
        return 'Medium Traffic'
    else:
        return 'High Traffic'

In [61]:
avg_trips_per_geohash['traffic_category'] = avg_trips_per_geohash['avg_daily_trips'].apply(categorize_traffic)

df_modified = df_modified.merge(
    avg_trips_per_geohash[['start_geohash', 'traffic_category']],
    on='start_geohash',
    how='left'
)

In [62]:
df_modified.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'member_casual', 'duration_min', 'geometry_start',
       'geometry_end', 'Zone_start_zone_name', 'Zone_start_GIS_ID',
       'Zone_start_OBJECTID', 'Zone_start_geometry', 'Zone_end_zone_name',
       'Zone_end_GIS_ID', 'Zone_end_OBJECTID', 'Zone_end_geometry', 'date',
       'Weather_temp', 'Weather_windspeedmean', 'Weather_sunrise',
       'Weather_sunset', 'Weather_conditions', 'Bikeshare_NAME_start',
       'Bikeshare_STATION_TYPE_start', 'Bikeshare_CAPACITY_start',
       'Bikeshare_REGION_ID_start', 'Bikeshare_REGION_NAME_start',
       'Bikeshare_geometry_start', 'Bikeshare_NAME_end',
       'Bikeshare_STATION_TYPE_end', 'Bikeshare_CAPACITY_end',
       'Bikeshare_REGION_ID_end', 'Bikeshare_REGION_NAME_end',
       'Bikeshare_geometry_end', 'StartStation_NAME', 'EndStation_NAME',
       'start_longitude', 'start_latitude', 'en

In [63]:
df_modified.to_csv('datasets/modified.csv', index=False)

PermissionError: [Errno 13] Permission denied: 'datasets/modified.csv'

In [64]:
def classify_weather(condition):
    if pd.isna(condition):
        return np.nan
    condition = condition.lower()
    if any(w in condition for w in ['clear', 'sunny']):
        return 'Sunny'
    elif any(w in condition for w in ['rain', 'storm', 'shower', 'drizzle']):
        return 'Rainy'
    elif any(w in condition for w in ['cloud', 'overcast', 'fog', 'mist']):
        return 'Cloudy'
    else:
        return 'Cloudy'

In [65]:
df_copy = df_modified.copy()

In [66]:
df_modified['weather_category'] = df_modified['Weather_conditions'].apply(classify_weather)

# Step 3: Drop old weather columns
df_modified.drop(columns=[
    'Weather_temp',
    'Weather_windspeedmean',
    'Weather_sunrise',
    'Weather_sunset',
    'Weather_conditions'
], inplace=True)

In [67]:
df_modified.drop(columns=[
    'Bikeshare_geometry_end',
    'Bikeshare_geometry_start',
    'geometry_end',
    'geometry_start'
], inplace=True)

In [68]:
df_modified

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,member_casual,duration_min,...,start_latitude,end_longitude,end_latitude,start_near_any_mall,end_near_any_mall,end_distance_to_closest_mall_m,start_geohash,end_geohash,traffic_category,weather_category
0,748A93D7DE8A41CD,classic_bike,2024-01-25 15:49:59,2024-01-25 15:52:35,1st & O St NW,31519,1st & L St NW,31677.0,member,2.600000,...,38.908643,-77.011987,38.903819,0,0,889.908267,dqcjre,dqcjrd,High Traffic,Rainy
1,75CBFD136F06305B,classic_bike,2024-01-02 16:44:58,2024-01-02 16:53:25,1st & O St NW,31519,4th & College St NW,31138.0,member,8.450000,...,38.908643,-77.018135,38.921233,0,0,2361.117404,dqcjre,dqcjrm,High Traffic,Cloudy
2,0536C9720F87E04C,classic_bike,2024-01-24 15:40:15,2024-01-24 15:43:55,1st & O St NW,31519,1st & L St NW,31677.0,member,3.666667,...,38.908643,-77.011987,38.903819,0,0,889.908267,dqcjre,dqcjrd,High Traffic,Cloudy
3,9E17390C218783B5,classic_bike,2024-01-04 15:35:00,2024-01-04 15:37:35,1st & O St NW,31519,1st & L St NW,31677.0,member,2.583333,...,38.908643,-77.011987,38.903819,0,0,889.908267,dqcjre,dqcjrd,High Traffic,Cloudy
4,00727D0E773CDFF7,electric_bike,2024-01-05 12:27:58,2024-01-05 12:35:40,1st & O St NW,31519,10th & G St NW,31274.0,casual,7.700000,...,38.908643,-77.026235,38.898243,0,1,296.143991,dqcjre,dqcjr1,High Traffic,Cloudy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,7F6EB9E56D972AD2,classic_bike,2024-01-05 22:58:07,2024-01-05 23:04:08,1st & O St NW,31519,11th & O St NW,31286.0,member,6.016667,...,38.908643,-77.027088,38.908431,0,0,842.684543,dqcjre,dqcjr5,High Traffic,Cloudy
93,C6FF417383F01E06,classic_bike,2024-01-13 12:45:26,2024-01-13 12:51:39,Alabama & MLK Ave SE,31800,Alabama Ave & Stanton Rd SE / Shops at Park Vi...,31813.0,casual,6.216667,...,38.843230,-76.981850,38.846270,0,0,3587.020946,dqchzz,dqckbp,Low Traffic,Rainy
94,8DD20FEDB618CE47,classic_bike,2024-01-08 19:11:35,2024-01-08 19:15:42,1st & O St NW,31519,11th & O St NW,31286.0,member,4.116667,...,38.908643,-77.027088,38.908431,0,0,842.684543,dqcjre,dqcjr5,High Traffic,Cloudy
95,03A98F39C412ACD9,classic_bike,2024-01-02 17:11:18,2024-01-02 17:23:10,11th & Park Rd NW,31651,11th & O St NW,31286.0,casual,11.866667,...,38.931322,-77.027088,38.908431,0,0,842.684543,dqcjrp,dqcjr5,Medium Traffic,Cloudy


In [70]:
df_modified['trip_price'] = df_modified.apply(hf.calculate_price, axis=1)

In [71]:
df_modified

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,member_casual,duration_min,...,end_longitude,end_latitude,start_near_any_mall,end_near_any_mall,end_distance_to_closest_mall_m,start_geohash,end_geohash,traffic_category,weather_category,trip_price
0,748A93D7DE8A41CD,classic_bike,2024-01-25 15:49:59,2024-01-25 15:52:35,1st & O St NW,31519,1st & L St NW,31677.0,member,2.600000,...,-77.011987,38.903819,0,0,889.908267,dqcjre,dqcjrd,High Traffic,Rainy,3.59
1,75CBFD136F06305B,classic_bike,2024-01-02 16:44:58,2024-01-02 16:53:25,1st & O St NW,31519,4th & College St NW,31138.0,member,8.450000,...,-77.018135,38.921233,0,0,2361.117404,dqcjre,dqcjrm,High Traffic,Cloudy,3.59
2,0536C9720F87E04C,classic_bike,2024-01-24 15:40:15,2024-01-24 15:43:55,1st & O St NW,31519,1st & L St NW,31677.0,member,3.666667,...,-77.011987,38.903819,0,0,889.908267,dqcjre,dqcjrd,High Traffic,Cloudy,3.59
3,9E17390C218783B5,classic_bike,2024-01-04 15:35:00,2024-01-04 15:37:35,1st & O St NW,31519,1st & L St NW,31677.0,member,2.583333,...,-77.011987,38.903819,0,0,889.908267,dqcjre,dqcjrd,High Traffic,Cloudy,3.59
4,00727D0E773CDFF7,electric_bike,2024-01-05 12:27:58,2024-01-05 12:35:40,1st & O St NW,31519,10th & G St NW,31274.0,casual,7.700000,...,-77.026235,38.898243,0,1,296.143991,dqcjre,dqcjr1,High Traffic,Cloudy,1.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,7F6EB9E56D972AD2,classic_bike,2024-01-05 22:58:07,2024-01-05 23:04:08,1st & O St NW,31519,11th & O St NW,31286.0,member,6.016667,...,-77.027088,38.908431,0,0,842.684543,dqcjre,dqcjr5,High Traffic,Cloudy,3.59
93,C6FF417383F01E06,classic_bike,2024-01-13 12:45:26,2024-01-13 12:51:39,Alabama & MLK Ave SE,31800,Alabama Ave & Stanton Rd SE / Shops at Park Vi...,31813.0,casual,6.216667,...,-76.981850,38.846270,0,0,3587.020946,dqchzz,dqckbp,Low Traffic,Rainy,1.00
94,8DD20FEDB618CE47,classic_bike,2024-01-08 19:11:35,2024-01-08 19:15:42,1st & O St NW,31519,11th & O St NW,31286.0,member,4.116667,...,-77.027088,38.908431,0,0,842.684543,dqcjre,dqcjr5,High Traffic,Cloudy,3.59
95,03A98F39C412ACD9,classic_bike,2024-01-02 17:11:18,2024-01-02 17:23:10,11th & Park Rd NW,31651,11th & O St NW,31286.0,casual,11.866667,...,-77.027088,38.908431,0,0,842.684543,dqcjrp,dqcjr5,Medium Traffic,Cloudy,1.00


In [72]:
df_modified['ended_at'] = pd.to_datetime(df_modified['ended_at'])

df_modified['payment_day'] = df_modified['ended_at'].dt.date

daily_revenue = df_modified.groupby(['payment_day', 'weather_category'])['trip_price'].sum().reset_index()

long_format = daily_revenue.copy()

wide_format = daily_revenue.pivot(index='payment_day', columns='weather_category', values='trip_price').fillna(0).reset_index()
wide_format.columns.name = None  # remove pivoted column name

print("Long Format:")
print(long_format.head())

print("\nWide Format:")
print(wide_format.head())

Long Format:
  payment_day weather_category  trip_price
0  2024-01-01            Rainy        3.59
1  2024-01-02           Cloudy        9.18
2  2024-01-03           Cloudy       20.45
3  2024-01-04           Cloudy       11.27
4  2024-01-05           Cloudy        8.68

Wide Format:
  payment_day  Cloudy  Rainy  Sunny
0  2024-01-01    0.00   3.59    0.0
1  2024-01-02    9.18   0.00    0.0
2  2024-01-03   20.45   0.00    0.0
3  2024-01-04   11.27   0.00    0.0
4  2024-01-05    8.68   0.00    0.0
